# Regression Exercise

# California Housing Data

This data set contains information about all the block groups in California from the 1990 Census. In this sample a block group on average includes 1425.5 individuals living in a geographically compact area.

The task is to aproximate the median house value of each block from the values of the rest of the variables.

It has been obtained from the LIACC repository. The original page where the data set can be found is: http://www.liaad.up.pt/~ltorgo/Regression/DataSets.html.

The Features:

housingMedianAge: continuous.

totalRooms: continuous.

totalBedrooms: continuous.

population: continuous.

households: continuous.

medianIncome: continuous.

medianHouseValue: continuous.


# The Data
Import the cal_housing_clean.csv file with pandas. Separate it into a training (70%) and testing set(30%).

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib as plt
from sklearn import model_selection

data  ==  pd.read_csvread_cs ('California Housing data.csv')
data = data.dropna(axis=0)
data.head()

In [ ]:
data.drop(labels='Unnamed: 0', inplace=True, axis=1)
data.head().describe()

In [ ]:
y_data = data['medianHouseValue']
data.drop(labels=['medianHouseValue'], inplace=True, axis =1)
x_data = data


X_trainX_train, X_test, y_train, y_test = model_selection.train_test_split(x_data, y_data, test_size=0.3, random_state=1)


y_trainy_train.head()

# Scale the Feature Data
Use sklearn preprocessing to create a MinMaxScaler for the feature data. Fit this scaler only to the training data. Then use it to transform X_test and X_train. Then use the scaled X_test and X_train along with pd.Dataframe to re-create two dataframes of scaled data.

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()
scaler.fit(X_train)

In [ ]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create Feature Columns
Create the necessary tf.feature_column objects for the estimator. They should all be trated as continuous numeric_columns.

In [ ]:
data.columns

In [ ]:
X_train = pd.DataFrame(data=X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test = pd.DataFrame(data=X_test_scaled, columns=X_test.columns, index=X_test.index)

In [ ]:
house_age = tf.feature_column.numeric_column('HouseAge')
av_rooms = tf.feature_column.numeric_column('AveRooms')
av_bedroom = tf.feature_column.numeric_column('AveBedrms')
population = tf.feature_column.numeric_column('Population')
av_occup = tf.feature_column.numeric_column('AveOccup')
med_income = tf.feature_column.numeric_column('MedInc')
longitude = tf.feature_column.numeric_column('Longitude')
latitude = tf.feature_column.numeric_column('Latitude')

feat_columns = [house_age, av_rooms, av_bedroom, population, av_occup, med_income, longitude, latitude]
feat_columns

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, batch_size=10,num_epochs=1000,shuffle=True)

In [ ]:
model  ==  tf.estimator .DNNRegressor(hidden_units=[10, 12, 14, 12, 10], feature_columns=feat_columns)

Train the model for ~1,000 steps. (Later come back to this and train it for more and check for improvement) 

In [ ]:
model.train(input_fn = input_func, steps = 10000)

Create a prediction input function and then use the .predict method off your estimator model to create a list or predictions on your test data.

In [ ]:
pred_input_func = tf.estimator.inputs.pandas_input_fn(x=X_test, batch_size=20,num_epochs=1,shuffle=False)

pred_model = model.predict(pred_input_func)
preds = list(pred_model)

In [ ]:
final_y_preds = []
for pred in preds:
    final_y_preds.append(pred['predictions'])

In [ ]:
final_y_preds[:20]

Calculate the RMSE. You should be able to get around 100,000 RMSE (remember that this is in the same units as the label.) Do this manually or use sklearn.metrics

In [ ]:
from sklearn import metrics
import math

rmse = math.sqrt(metrics.mean_squared_error(y_test, final_y_preds))
rmse